In [1]:
import pandas as pd
import numpy as np

#import seaborn as sns
from matplotlib import pyplot as plt


In [2]:
df = pd.read_csv('data/data.csv')
print(f'${len(df)} lines loaded')


$11914 lines loaded


In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')
df.head(5)

,make,model,year,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,market_category,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp
0,bmw,1_series_m,2011,premium_unleaded_(required),335.0,6.0,manual,rear_wheel_drive,2.0,"factory_tuner,luxury,high-performance",compact,coupe,26,19,3916,46135
1,bmw,1_series,2011,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,convertible,28,19,3916,40650
2,bmw,1_series,2011,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,high-performance",compact,coupe,28,20,3916,36350
3,bmw,1_series,2011,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,coupe,28,18,3916,29450
4,bmw,1_series,2011,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,luxury,compact,convertible,28,18,3916,34500


In [4]:
n = len(df)
n_test = int(0.2*n)
n_val = int(0.2*n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test = df_shuffled.iloc[n_train+n_val:].copy()

y_train = np.log1p(df_train.msrp.values)

del df_train['msrp']

In [5]:
def linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [9]:
base = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'popularity']

In [10]:
def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [11]:
X_train = prepare_X(df_train)
w_0, w = linear_regression(X_train, y_train)

In [12]:
y_pred = w_0 + X_train.dot(w)

In [13]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [14]:
rmse(y_train, y_pred)

0.7493754559677959

In [15]:
dataframe = pd.DataFrame(X_train,columns=base)
dataframe['predicted_msrp'] = y_pred
dataframe['actual_msrp'] = y_train
dataframe.head(5)


,engine_hp,engine_cylinders,highway_mpg,city_mpg,popularity,predicted_msrp,actual_msrp
0,237.0,4.0,34.0,24.0,113.0,10.441993,10.894904
1,250.0,6.0,21.0,16.0,640.0,9.961225,10.382699
2,278.0,6.0,32.0,21.0,2202.0,10.437488,10.440741
3,365.0,6.0,21.0,15.0,5657.0,10.939716,10.964675
4,284.0,6.0,27.0,20.0,2009.0,10.407605,10.403687
